# Project description "Continuous control"

The "Continuous Control" project is one of three problems that are part of [Udacity](https://www.udacity.com/)'s Deep Reinforcement Learning Nanodegree. The task is to apply torque to the two joints of a robotic arm, in order to move a hand at the end of the arm into a dynamically moving target zone.

A description of the environment can be found in [0]; the agent has a 33 dimensional observation space that describes both the position and movement of the arm as well as the location of the target zone. The action space consists of four continuous torque values, two values per joint of the arm. A reward is given when the hand is in the target zone.

My first attempts to solve the task using the Deep Deterministic Policy Gradient algorithm (DDPG, Lillicrap et al., 2015, [1]) were unsuccesful. [Spinning Up in Deep RL](https://spinningup.openai.com/en/latest/) [2] has a short description of various DRL algorithms, which is where I found an improvement of DDPG: the Twin Delayed DDPG algorithm (TD3, Fujimoto et al., 2018, [3]) has some improvements over DDPG that helped me to solve the task.

## Actor Critic Methods, DDPG & T3D algorithm
Techniques like DQN learning work well with discrete and relatively small action spaces, however in continuous action spaces, finding the optimal value of the Q-value function is an optimization problem on its own [1]. Lillicrap et al. (2015, [1]) solved this problem by using an actor-critic method. In the actor-critic method a policy (actor) is directly learned with help of the Q-value function (critic). 

While the original manuscripts are definitely worth reading, I will briefly mention the main ideas following the summaries given by "Spinning Up Deep RL" (see [2] for their DDPG description and [3] for TD3):

Deep Deterministic Policy Gradient learns a representation $\phi_{local}$ of the action-value function Q very similar to the DQN algorithm by using the Bellman equation. In order to improve stability another representation of the action value function $Q_{target}$ is used, and this neural network is slowly updated to match $Q_{local}$. The target action-value function $Q_{target}$ can be updated with help of the the Bellman equation:

$$Q_{target}(s_{0}, a_{0})=r + \gamma * Q_{target}(s_{1}, \mu_{\phi_{target}}(s_{1}))$$ 
where $s_{0}$ is the current state, $a_{0}$ the current action, r the reward after taking the action and $s_{1}$ is the next sate.

The local network is then updated by minimizing the mean squared error between $Q_{target}(s_{0}, a_{0})$ and $Q_{local}(s_{0}, a_{0})$.

The representation of the policy $\mu_{\theta}$ is learned by taking gradient ascent steps in the direction of the gradient of the expected action-value function [2]:
$$ \nabla_{\theta}Q_{\phi}(s_{0}, \mu_{\theta}(s_{0}))$$
As explained in [3] and [4] DDPG can be instable due to an overestimation of the learned Q-values. 
The T3D algorithm by Fujimoto et al. (2018, [3]) adds three improvements to the DDPG method that help stabilize the algorithm's performance:
- The algorithm uses two Q-functions $Q_{A}$ and $Q_{B}$. The minimum of the predictions of the Q-value, $min(Q_{A}\mu_{\phi_{target}}(s_{1}), Q_{B}\mu_{\phi_{target}}(s_{1}))$ is used during the Bellman update of the critic neural network. This helps prevent overestimation of the Q-value function during training.
- The actor (or policy) network is updated less frequently than the critic networks
- Noise is added to the target action in the critic update step, which helps, for example, prevent problems due to spikes in the learned Q-value function.

## Implementation

I started implementing DDPG by loosely following the [DDPG pendulum implementation by Udacity](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum) [7].
Similarly to the Udacity benchmark implementation, the neural networks were not updated in each step in order to increase stability. 10 update steps with a batch size of 64 were performed after 20 steps in the environment.
Rather than performing the step and experience handling in the main loop, I implemented an "episode generator", following an idea from [8]. The advantage of the episode generator is that it can later easily be extended to return n-step updates rather than one step updates.

Since the DDPG implementation did not lead to a learning agent I started implementing the improvements of the TD3 algorithm ([3], [4]): double Q-learning, delayed policy updates and target policy smoothing. Similar to the [original author's implementation](https://github.com/sfujim/TD3) ([5]), I implemented the two critic networks in a single class that returns Q values for both in one step.

## Results

A short animation of the trained agent is shown below:  
![Recher Gif](results/reacher.gif)


The TD3 algorithm was able to learn. Fig. 1 shows the scores of each individual episode (blue curve) as well as a running mean over 100 episodes. The average score was above 30 after roughly 500 episodes. Results for other runs with slightly different parameters can be found in the results folder. The settings are listed below and are pretty standard settings from the DDPG manuscript [1] and the TD3 manuscript [4].
One noteworthy difference was the relatively low learning rates of 5e-4 for both the actor and the critic network.![Learning curve](results/run003/learning_curve.png)


Figure 2 and 3 show the actor-loss and critic-loss. The values presented are an average loss of the 10 update steps at the end of an episode. The actor-loss steadily increases during the training. Since it represents the expected Q-value (or the expected reward from a given state) it is expected to be increasing.
![Actor Loss](results/run003/a_loss.png)

A little more suprising, the critic-loss which is the mean squared error between the Bellman target Q-value and the expected Q-value does show large values with increasing training time, it should  decrease with training time. In Figure 3 the individual critic losses for the critic networks $Q_{A}$ (blue) and $Q_{B}$ (orange) are shown, however the values $Q_{A}$ are hidden behind $Q_{B}$. The peaks of higher loss function values are not equidistant (in particular they did not always appear at the end of an episode). ![Critic Loss](results/run003/c_loss.png)
The x-label on the critic-loss Figure above is mislabeled, it should be step number instead of episode number.

## Future improvements

One obvious improvement would be to further investigate the cause of the increasing critic-loss. Since the implementation uses two Q-target networks one approach would be to look at the loss of each network individually. 

Besides this (necessary) bug-fix there are many other interesting ways to improve the agent. Distributed Distributional Deterministic Policy Gradients (D4PG, [6]) makes use of a complete distribution of reward values instead of only a mean value of the reward. It would also be interesting to combine D4PG with the improvements from T3D.
Additionally using n-step updates rather than just the next step would be an interesting direction that could improve the agent.

## Settings for the presented run:
The settings for the run shown above are printed below:

In [ ]:
settings = {
    'batch_size': 64,            # Number of experience samples per training step
    'buffer_size': int(3e6),     # Max number of samples in the replay memory
    'gamma': 0.99,               # Reward decay factor
    'tau': 1e-3,                 # Update rate for the slow update of the target networks
    'lr_actor': 5e-4,            # Actor learning rate
    'lr_critic': 5e-4,           # Critic learning rate
    'action_noise': 0.4,         # Noise added during episodes played
    'action_clip': 1.0,          # Actions are clipped to +/- action_clip
    'target_action_noise': 0.4,  # Noise added during the critic update step
    'target_noise_clip': 0.2,    # Noise clip for the critic update step
    'number_steps': 1,           # Number of steps for roll-out, currently not used
    'optimize_critic_every': 2,  # Update the critic only every X update steps
    'pretrain_steps': int(10000),# Number of random actions played before training starts
    'actor_settings': actor_settings,
    'critic_settings': critic_settings}

## Further reading and references

Since my original DDPG implementation did not converge, I started reading various different sources, which are listed below:  
- As suggested by Udacity, I started by studying the DDPG pendulum implementation [7]
- The descriptions of DDPG and TD3 from OpenAi ([3], [4]) were great to understand both algorithms in more detail
- The TD3 implementation of the original authors can be found under [5]
- The Deep Reinforcement Learning Hands-On book by Maxim Lapan [8] was a valuable source that greatly helped me understand the topic better and the hands-on code examples were really helpful to see, how things can be implemented in Python and PyTorch.
- One Udacity student helped many people in the internal discussion boards and shared his github profile (which you can find at [9]). The github profile features a very thorough implementation of the DP4G algorithm. The idea of using a pre-train phase with random actions before training is from this implementation.  
- TowardsDataScience had a series of blogposts on the same task [10]. The post features implementations of three different algorithms: DDPG, DP4G and A2C.
- The PyTorch tutorials and documentation were again very helpful [11]

# References
[0] https://github.com/udacity/deep-reinforcement-learning/tree/master/p2_continuous-control  
[1] Lillicrap, Timothy P., et al. "Continuous control with deep reinforcement learning." arXiv preprint arXiv:1509.02971 (2015).  
[2] https://spinningup.openai.com/en/latest/algorithms/ddpg.html  
[3] https://spinningup.openai.com/en/latest/algorithms/td3.html  
[4] Fujimoto, Scott, Herke van Hoof, and David Meger. "Addressing function approximation error in actor-critic methods." arXiv preprint arXiv:1802.09477 (2018).  
[5] https://github.com/sfujim/TD3, last accessed: 2019-04-14  
[6] Barth-Maron, Gabriel, et al. "Distributed distributional deterministic policy gradients." arXiv preprint arXiv:1804.08617 (2018).  
[7] https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum, last accessed: 2019-04-14  
[8] Lapan, Maxim. Deep Reinforcement Learning Hands-On: Apply modern RL methods, with deep Q-networks, value iteration, policy gradients, TRPO, AlphaGo Zero and more. Packt Publishing Ltd, 2018.  
[9] https://github.com/whiterabbitobj/Continuous_Control/, last accessed: 2019-04-14  
[10] https://towardsdatascience.com/rl-train-the-robotic-arm-to-reach-a-ball-part-01-1cecd2e1cfb8  and 
     https://towardsdatascience.com/rl-train-the-robotic-arm-to-reach-a-ball-part-02-fc8822ace1d8  
[11] https://pytorch.org/